In [1]:
from labic_images_segmentation import Dataset, DataAugmentation, SegmentationModel, SaveReport
import tensorflow as tf
from segmentation_models import Unet, Linknet

from tensorflow.keras.optimizers import Adam
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

Segmentation Models: using `keras` framework.


In [11]:
dataset_folder = "./TM40_Original"
norm_imgs_folder = "/Norm_images/*"
gt_folder = "/GT_images/*"
NEW_SIZE = 128
output_folder = "./TM40_46Prod"
gt_test_folder = "/GT_Producao/*"
n_fold = 2
batch_size = 4
epochs = 2

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=50)

In [3]:
imgs = Dataset(folder=dataset_folder, norm_imgs_folder=norm_imgs_folder, gt_folder=gt_folder, NEW_SIZE=NEW_SIZE)


Imagens carregadas com sucesso.


In [4]:
imgs.split_dataset()


Dataset subdividido.
20.0% dos dados para validação.
80.0% dos dados para treino.


In [5]:
data_aug = DataAugmentation(imgs.X_train, imgs.Y_train, imgs.X_val, imgs.Y_val)


Etapa de Data Augmentation concluída.


In [6]:
segment = SegmentationModel(X_train=imgs.X_train, trainDS=data_aug.trainDS, valDS=data_aug.valDS, epochs=epochs, callback=early_stopping)

Epoch 1/2
233/233 [==============================] - 20s 51ms/step - loss: 1.1802 - iou_score: 0.0436 - val_loss: 1.1340 - val_iou_score: 0.0370
Epoch 2/2
233/233 [==============================] - 10s 42ms/step - loss: 1.1072 - iou_score: 0.0443 - val_loss: 1.1143 - val_iou_score: 0.0440


In [7]:
save_report = SaveReport(model=segment.model, folder_name=output_folder, n_fold=n_fold, use_batch_size=batch_size, epochs=epochs)

Diretorio  ./TM40_46Prod/outputs/Exec_2023-04-15-20-18-08.781914  ja existe
Diretorio  ././TM40_46Prod/outputs/Exec_2023-04-15-20-18-08.781914/fold_2  ja existe

Modelo salvo.
Nome: 4_2_exec_2023-04-15-20-18-08.781914_fold_2
Salvo em: ./TM40_46Prod/outputs/Exec_2023-04-15-20-18-08.781914


In [10]:
save_report.imgs_to_predict

'/outputs/Exec_2023-04-15-20-18-08.781914'

In [12]:
test_imgs = Dataset(folder=output_folder, norm_imgs_folder=save_report.imgs_to_predict, gt_folder=gt_test_folder, NEW_SIZE=NEW_SIZE)

Algo está errado com as imagens


ValueError: Could not find a format to read the specified file in single-image mode